In [17]:
companies = ['BNP','AstraZeneca','Bayer','BT Group','Coca-cola','Danone','Ford','Oreal','Microsoft','Philips'] # ,'Schneider','Toyota'

In [2]:
lesechos = {
    "main_page": "https://www.lesechos.fr/",
    "titres": {}
}
lemondediplo = {
    "main_page": "https://www.monde-diplomatique.fr/",
    "titres": {},
    "exclure": ["Le journal","Services","À propos","Le Monde diplomatique"]
}

## Selenium Part

In [3]:
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [8]:
driver = webdriver.Firefox()

### Le monde diplomatique

In [9]:
url = lemondediplo['main_page']
driver.get(url)

In [10]:
def fetch_info_company(company, url, wait_time = 3):
    element = driver.find_element(By.ID, "recherche").send_keys(company)
    element = driver.find_element(By.ID, "recherche").send_keys(Keys.ENTER)

    time.sleep(wait_time)

    elements = driver.find_elements(By.CSS_SELECTOR, "h3")

    lemondediplo['titres'][company] = []

    for e in elements:
        lemondediplo['titres'][company].append(e.text)

    i = 0

    while True:
        i = i + 10
        try:
            driver.find_element(By.LINK_TEXT, f"{i}").send_keys(Keys.ENTER)
        except:
            break
        
        time.sleep(wait_time)

        elements = driver.find_elements(By.CSS_SELECTOR, "h3")

        for e in elements:
            if not e in lemondediplo["exclure"]:
                lemondediplo['titres'][company].append(e.text)

    driver.get(url)

    time.sleep(wait_time)

    # print(lemondediplo['titres']['BNP'])

    return True

In [21]:
lemondediplo['titres']['BNP']

['BNP-Paribas, une affaire de géométries variables',
 'Les Etats-Unis mettent les banques à l’amende',
 'Crispations chez les promoteurs du GMT',
 'Banques, bonus, bénéfices : le paradoxe des parasites',
 'Cigarettes, bakchichs et sodas',
 'Fusions d’entreprises, festins de prédateurs',
 'La flambée des privatisations',
 'Les quatre degrés de l’Apocalypse',
 'Est-ce vraiment la fin du dollar ?',
 'Horreur, stupeur, terreur… Bonheur',
 'Les liaisons dangereuses de banques françaises avec la colonisation israélienne',
 'Bloody Nasty People',
 'Banqueroute. Comment Fortis a ébranlé la Belgique',
 'La politique du capital',
 'Banques sur le gril',
 'Bâtir aussi. Fragments d’un monde révolutionné',
 'Deux écologistes face à Bruxelles',
 'La Panne. Repenser le travail et changer la vie. Entretien avec Béatrice Bouniol',
 '« Le Monde diplomatique » en bande dessinée',
 'Le journal',
 'Services',
 'À propos',
 'Le Monde diplomatique',
 'La mine d’or de l’e-santé',
 'Une « petite » association 

In [ ]:
for company in companies:
    fetch_info_company(company,url)

### Les echos

Ce site est un petit peu plus difficile à scrapper, nous nous sommes pour l'instant concentrés sur le monde diplomatique.

In [31]:
url = lesechos['main_page']
driver.get(url)

In [36]:
element = driver.find_element(By.CLASS_NAME, "sc-14kwckt-28").send_keys("BNP") # sc-14kwckt-28 sc-ywv8p0-0 sc-166k8it-1 wwuyu jCZKki cQRcWN

In [37]:
element = driver.find_element(By.CLASS_NAME, "sc-14kwckt-28").send_keys(Keys.ENTER)

In [ ]:
elements = driver.find_elements(By.CSS_SELECTOR, "h3")
elements

## Dataframe creation

In [18]:
import pandas as pd

In [19]:
df = pd.DataFrame({
    'newspaper': ['lemondediplo' for i in range(len(companies))],
    'company': companies,
    'title': [lemondediplo['titres'][company] for company in companies]
})

df = df.explode(column='title', ignore_index= True)

df.head()

,newspaper,company,title
0,lemondediplo,BNP,"BNP-Paribas, une affaire de géométries variables"
1,lemondediplo,BNP,Les Etats-Unis mettent les banques à l’amende
2,lemondediplo,BNP,Crispations chez les promoteurs du GMT
3,lemondediplo,BNP,"Banques, bonus, bénéfices : le paradoxe des pa..."
4,lemondediplo,BNP,"Cigarettes, bakchichs et sodas"


In [20]:
df.to_csv('article_titles.csv')